## Initial data solver using relaxation

We try to solve the time symmetric initial data when given a $ρ$ field on a cube. 
We put Robin boudary conditions.

Thus we shall be solving:

$$
ψ_{tt} = \Delta ψ + ψ^5 ρ \;\;\;\; \vec{x}\cdot \nabla ψ + (1-ψ) = 0
$$

Using second order finite differences inside the domain and first order at the boudaries we get for the boundary values (which are not evolved):

For boundary conditions we shall put an outgoing wave, but taking into account that we want a radial wave and that something which at first order should be a monopolar field.

$$
ψ_t = (x^i\partial_i ψ - (ψ - ψ_0))/r
$$ 
Same for the time derivative, but that part should be unnessary becuse this should decay to a static solution.




In [7]:
using SummationByPartsOperators
using Plots
using Revise
using Distributions
using FFTW
using LinearAlgebra
using Base.Threads
using HDF5
 
includet("../PIC/PIC-1D/aux_functions/aux_functions.jl")
includet("local_aux_functions.jl")


In [8]:
J = (45,45,45)
Box_x = (-22.0, 22.0, -22.0, 22.0, -22.0, 22.0)

Dx = derivative_operator(MattssonNordström2004(), derivative_order = 1, accuracy_order = 2, xmin = Box_x[1], xmax = Box_x[2], N=J[1])
Dy = derivative_operator(MattssonNordström2004(), derivative_order = 1, accuracy_order = 2, xmin = Box_x[3], xmax = Box_x[4], N=J[2])
Dz = derivative_operator(MattssonNordström2004(), derivative_order = 1, accuracy_order = 2, xmin = Box_x[5], xmax = Box_x[6], N=J[3])
D2x = derivative_operator(MattssonNordström2004(), derivative_order = 2, accuracy_order = 2, xmin = Box_x[1], xmax = Box_x[2], N=J[1])
D2y = derivative_operator(MattssonNordström2004(), derivative_order = 2, accuracy_order = 2, xmin = Box_x[3], xmax = Box_x[4], N=J[2])
D2z = derivative_operator(MattssonNordström2004(), derivative_order = 2, accuracy_order = 2, xmin = Box_x[5], xmax = Box_x[6], N=J[3])
x = collect(SummationByPartsOperators.grid(Dx))
y = collect(SummationByPartsOperators.grid(Dy))
z = collect(SummationByPartsOperators.grid(Dz));



In [ ]:
dx = differentials(Box_x, J, periodic=false)

In [10]:
n_fields = 2 # the field and its derivative 
u = zeros(n_fields,J...)
du = similar(u)
d2 = zeros(J...);
dxu = zeros(J...);
dyu = zeros(J...);
dzu = zeros(J...);

In [ ]:

x0 = [0.0,0.0,0.0]
r0 = 10.0
p = 4
par = (x,y,z,x0,Box_x, r0, p, J)

#ρ = zeros(J...)

ρ =  get_source(chichon, par);
#heatmap(ρ[:,:,20],aspectratio=1)
#@show norm(ρ)
maximum(abs.(ρ))

In [ ]:
T = 40.0
M = 400
dt = T/(M-1)
t = 0.0

#null initial data

u[:,:,:,:] .= 0.0

par = (1.0, 1.0, 20.0 )
par_F = (x,y,z,dxu,dyu,dzu,Dx,Dy,Dz,D2x,D2y,D2z,d2,dx,ρ,J, par)

norms = zeros(2,M)

for m in 2:M
    RK4_Step!(F,u,t,dt,par_F)
    t = t + dt
    norms[:,m] = [norm(u[1,:,:,:]),norm(u[2,:,:,:])]
    println("norm u_1 = $(norms[1,m]) norm u_2 = $(norms[2,m])")
end
    

In [ ]:
plot(norms[1,:], label="phi_norm")
plot!(norms[2,:], label="phi_t_norm")

In [ ]:
surface(x,y,u[1,:,:,(J[3]+1)÷2])

In [ ]:
plot(x,u[1,:,(J[2]+1)÷2,(J[3]+1)÷2])

In [ ]:
plot(x,u[2,:,(J[2]+1)÷2,(J[3]+1)÷2])